In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_ncav = pd.read_csv('../datasets/data_value_ncav.csv', dtype={'거래소코드':'object'}, index_col=0)
data_ncav.shape

(8901, 8)

In [3]:
data_ncav.head()

,year,회사명,거래소코드,회계년도,시가총액(백만원),유동자산(백만원),부채(백만원),당기순이익(백만원)
5360,2007,평산차업집단유한공사,950010,2006/09,1.746000e+04,52122.0,45241.0,10777.0
5384,2008,삼성전자(주),005930,2007/12,6.643200e+07,40363068.0,37403228.0,7922981.0
5385,2008,포스코홀딩스(주),005490,2007/12,3.313100e+07,14291551.0,11157023.0,3677964.0
5386,2008,한국전력공사(주),015760,2007/12,1.899040e+07,9055041.0,38661739.0,1467168.0
5387,2008,에스케이텔레콤(주),017670,2007/12,1.696949e+07,4776689.0,7361302.0,1562265.0


In [4]:
data_ncav.tail()

,year,회사명,거래소코드,회계년도,시가총액(백만원),유동자산(백만원),부채(백만원),당기순이익(백만원)
17333,2023,제이준코스메틱(주),025620,2022/12,24263.016860,22336.0,6806.0,-21703.0
17334,2023,(주)대유플러스,000300,2022/12,20865.059985,254616.0,431958.0,-24286.0
17335,2023,비케이탑스(주),030790,2022/12,19208.323080,68846.0,89910.0,-14400.0
17336,2023,(주)코아스,071950,2022/12,18637.047630,30927.0,58376.0,-1663.0
17337,2023,일정실업(주),008500,2022/12,15480.000000,13595.0,27888.0,-8534.0


In [5]:
# 결측치 없음

data_ncav.isna().sum()

year          0
회사명           0
거래소코드         0
회계년도          0
시가총액(백만원)     0
유동자산(백만원)     0
부채(백만원)       0
당기순이익(백만원)    0
dtype: int64

### (가치주) - NCAV 전략 구현

- 데이터프레임명 : `data_ncav`

- 파생변수 생성
    - `NCAV` = 유동자산(백만원) - 부채(백만원)
    - `NCAV/시총` = NCAV / 시가총액(백만원)
- 조건식 생성
    - `조건1` : `NCAV/시총` > 1 (순유동자산 > 시가총액)
    - `조건2` : `당기순이익(백만원)` > 0 (흑자기업)

In [6]:
data_ncav.head()

,year,회사명,거래소코드,회계년도,시가총액(백만원),유동자산(백만원),부채(백만원),당기순이익(백만원)
5360,2007,평산차업집단유한공사,950010,2006/09,1.746000e+04,52122.0,45241.0,10777.0
5384,2008,삼성전자(주),005930,2007/12,6.643200e+07,40363068.0,37403228.0,7922981.0
5385,2008,포스코홀딩스(주),005490,2007/12,3.313100e+07,14291551.0,11157023.0,3677964.0
5386,2008,한국전력공사(주),015760,2007/12,1.899040e+07,9055041.0,38661739.0,1467168.0
5387,2008,에스케이텔레콤(주),017670,2007/12,1.696949e+07,4776689.0,7361302.0,1562265.0


In [7]:
# 파생변수 생성
data_ncav['NCAV'] = data_ncav['유동자산(백만원)'] - data_ncav['부채(백만원)']
data_ncav['NCAV/시총'] = data_ncav['NCAV'] / data_ncav['시가총액(백만원)']

In [8]:
# 조건1: ncav/시총 > 1
data_ncav['조건1'] = (data_ncav['NCAV/시총'] > 1)

# 연도별 (조건1)에 맞는 종목 수 출력
data_ncav[data_ncav['조건1']]['year'].value_counts()

2023    61
2022    49
2019    36
2020    36
2008    35
2018    33
2011    30
2012    25
2021    24
2017    19
2009    14
2013    14
2016    13
2010    12
2014     9
2015     8
Name: year, dtype: int64

In [9]:
# 조건2: 흑자기업(당기순이익>0)
data_ncav['조건2'] = data_ncav['당기순이익(백만원)'] > 0

In [10]:
# 조건1 & 조건2
data_ncav['최종조건'] = data_ncav['조건1'] & data_ncav['조건2']

# 연도별 (조건1&조건2)에 맞는 종목 수 출력
data_ncav[data_ncav['최종조건']]['year'].value_counts()

2023    52
2022    44
2019    32
2008    31
2020    31
2018    29
2011    26
2021    20
2012    19
2017    18
2016    13
2009    12
2010    12
2013     9
2014     7
2015     7
Name: year, dtype: int64

In [11]:
data_ncav.reset_index(drop=True, inplace=True)
data_ncav

,year,회사명,거래소코드,회계년도,시가총액(백만원),유동자산(백만원),부채(백만원),당기순이익(백만원),NCAV,NCAV/시총,조건1,조건2,최종조건
0,2007,평산차업집단유한공사,950010,2006/09,1.746000e+04,52122.0,45241.0,10777.0,6881.0,0.394101,False,True,False
1,2008,삼성전자(주),005930,2007/12,6.643200e+07,40363068.0,37403228.0,7922981.0,2959840.0,0.044554,False,True,False
2,2008,포스코홀딩스(주),005490,2007/12,3.313100e+07,14291551.0,11157023.0,3677964.0,3134528.0,0.094610,False,True,False
3,2008,한국전력공사(주),015760,2007/12,1.899040e+07,9055041.0,38661739.0,1467168.0,-29606698.0,-1.559035,False,True,False
4,2008,에스케이텔레콤(주),017670,2007/12,1.696949e+07,4776689.0,7361302.0,1562265.0,-2584613.0,-0.152309,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8896,2023,제이준코스메틱(주),025620,2022/12,2.426302e+04,22336.0,6806.0,-21703.0,15530.0,0.640069,False,False,False
8897,2023,(주)대유플러스,000300,2022/12,2.086506e+04,254616.0,431958.0,-24286.0,-177342.0,-8.499472,False,False,False
8898,2023,비케이탑스(주),030790,2022/12,1.920832e+04,68846.0,89910.0,-14400.0,-21064.0,-1.096608,False,False,False
8899,2023,(주)코아스,071950,2022/12,1.863705e+04,30927.0,58376.0,-1663.0,-27449.0,-1.472819,False,False,False


### 포트폴리오 구성
- [참고사이트](https://drshininvestment.tistory.com/m/28)
- NCAV 전략
    - NCAV/시가총액 > 1.5
    - 흑자기업 : 당기순이익 > 0

- 완화 NCAV 전략
    - NCAV/시가총액 > **1**
    - 흑자기업
    - 최대 20 종목
        - NCAV/시가총액 높은 순

In [12]:
# 최종조건(NCAV/시총>1 and 흑자기업)에 맞는 데이터만 남기고 모두 제거
df_ncav = data_ncav[data_ncav['최종조건']].reset_index(drop=True)
df_ncav.shape

(362, 13)

In [13]:
# 연도별 포트폴리오 편입 가능 종목 수
df_ncav['year'].value_counts()

2023    52
2022    44
2019    32
2008    31
2020    31
2018    29
2011    26
2021    20
2012    19
2017    18
2016    13
2009    12
2010    12
2013     9
2014     7
2015     7
Name: year, dtype: int64

In [14]:
years = sorted(df_ncav['year'].unique())

portfolios = []
for year in years:
    # print(year)
    df_by_year = df_ncav[(df_ncav['year']==year)]
    df_by_year = df_by_year.sort_values('NCAV/시총', ascending=False) # 내림차순 정렬
    df_by_year = df_by_year.head(20) # NCAV/시총 높은 순서대로 최대 20개 추출
    
    # 시가총액 기준 투자 비중 계산
    df_by_year['pf_ratio'] = df_by_year['시가총액(백만원)'] / df_by_year['시가총액(백만원)'].sum()
    
    portfolios.append(df_by_year)
    
df_portfolios = pd.concat(portfolios).reset_index(drop=True)
# df_portfolios = df_portfolios.loc[:, :'pf_ratio']
df_portfolios

,year,회사명,거래소코드,회계년도,시가총액(백만원),유동자산(백만원),부채(백만원),당기순이익(백만원),NCAV,NCAV/시총,조건1,조건2,최종조건,pf_ratio
0,2008,(주)동성화학,005190,2007/12,6.766614e+03,136542.0,91447.0,15021.0,45095.0,6.664337,True,True,True,0.006144
1,2008,(주)씨앤우방,013200,2007/12,6.561833e+03,455492.0,423375.0,26.0,32117.0,4.894517,True,True,True,0.005959
2,2008,KISCO홀딩스(주),001940,2007/12,9.876600e+04,629813.0,333068.0,73488.0,296745.0,3.004526,True,True,True,0.089685
3,2008,(주)에스와이코퍼레이션,008080,2007/12,9.438399e+03,55806.0,28970.0,1954.0,26836.0,2.843279,True,True,True,0.008571
4,2008,(주)윌비스,008600,2007/12,2.430670e+04,118460.0,60169.0,5922.0,58291.0,2.398145,True,True,True,0.022072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,2023,태광산업(주),003240,2022/12,6.791740e+05,1928389.0,702185.0,342230.0,1226204.0,1.805434,True,True,True,0.119891
253,2023,(주)대덕,008060,2022/12,2.165581e+05,803619.0,419083.0,133396.0,384536.0,1.775672,True,True,True,0.038228
254,2023,케이피엑스홀딩스(주),092230,2022/12,2.167243e+05,812718.0,448980.0,37511.0,363738.0,1.678344,True,True,True,0.038257
255,2023,(주)유수홀딩스,000700,2022/12,1.507821e+05,325176.0,79255.0,44119.0,245921.0,1.630970,True,True,True,0.026617


In [15]:
# 연도별 NCAV 포트폴리오 편입 종목 수
df_portfolios['year'].value_counts()

2008    20
2011    20
2018    20
2019    20
2020    20
2021    20
2022    20
2023    20
2012    19
2017    18
2016    13
2009    12
2010    12
2013     9
2014     7
2015     7
Name: year, dtype: int64

In [16]:
df_portfolios[df_portfolios['year']==2015]

,year,회사명,거래소코드,회계년도,시가총액(백만원),유동자산(백만원),부채(백만원),당기순이익(백만원),NCAV,NCAV/시총,조건1,조건2,최종조건,pf_ratio
99,2015,KISCO홀딩스(주),001940,2014/12,229846.16720,659500.0,250328.0,14866.0,409172.0,1.780199,True,True,True,0.166780
100,2015,(주)에스제이엠홀딩스,025530,2014/12,86766.58648,158951.0,34915.0,16921.0,124036.0,1.429536,True,True,True,0.062959
101,2015,동일산업(주),004890,2014/12,137994.73350,236657.0,41048.0,11211.0,195609.0,1.417511,True,True,True,0.100131
102,2015,만호제강(주),001080,2014/06,74492.50000,149594.0,45901.0,5273.0,103693.0,1.391992,True,True,True,0.054053
103,2015,(주)신도리코,029530,2014/12,502489.44565,681360.0,95686.0,17115.0,585674.0,1.165545,True,True,True,0.364614
104,2015,CS홀딩스(주),000590,2014/12,91550.42260,134005.0,35962.0,14362.0,98043.0,1.070918,True,True,True,0.066430
105,2015,삼영전자공업(주),005680,2014/12,255000.00000,289402.0,28600.0,18510.0,260802.0,1.022753,True,True,True,0.185032


In [17]:
df_portfolios.groupby('year')['pf_ratio'].sum()

year
2008    1.0
2009    1.0
2010    1.0
2011    1.0
2012    1.0
2013    1.0
2014    1.0
2015    1.0
2016    1.0
2017    1.0
2018    1.0
2019    1.0
2020    1.0
2021    1.0
2022    1.0
2023    1.0
Name: pf_ratio, dtype: float64

In [18]:
# 파일로 저장
df_portfolios.to_csv('../datasets/portfolio_ncav.csv')